In [5]:
from dotenv import load_dotenv
load_dotenv()
import os
from groq import Groq
from pinecone import Pinecone, ServerlessSpec
from sentence_transformers import SentenceTransformer


c:\Users\Manizha khorram\miniconda3\envs\rag\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [14]:
pc = Pinecone(api_key= os.getenv('PINECONE_API_KEY'))
pc.create_index(
    name = "rag", dimension=1024 ,metric='cosine' , spec=ServerlessSpec(cloud='aws', region='us-east-1')
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '89741a7a7330c196888e241ee2b66292', 'Date': 'Thu, 22 Aug 2024 00:34:10 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [2]:
import json
data = json.load(open('reviews.json'))
data['reviews']

[{'professor': 'Dr. Emily Sharma',
  'subject': 'Computer Science',
  'stars': 4,
  'review': 'Dr. Sharma is an excellent professor. Her lectures are engaging, and she is always willing to help students during office hours. Highly recommended!'},
 {'professor': 'Prof. John Williamson',
  'subject': 'Economics',
  'stars': 3,
  'review': "Prof. Williamson's class was challenging, but I learned a lot. He could be a bit strict, but I appreciated his high standards."},
 {'professor': 'Dr. Olivia Martinez',
  'subject': 'Biology',
  'stars': 5,
  'review': "Dr. Martinez is hands down the best biology professor I've had. Her passion for the subject is contagious, and she makes the material easy to understand."},
 {'professor': 'Prof. Michael Chen',
  'subject': 'Mathematics',
  'stars': 2,
  'review': "I struggled a lot in Prof. Chen's class. His lectures were dry, and he didn't seem interested in helping students who were falling behind."},
 {'professor': 'Dr. Sarah Nguyen',
  'subject': 'E

In [11]:


# Load the correct multilingual E5 model
model = SentenceTransformer('intfloat/multilingual-e5-large')

processed_data = []

for review in data['reviews']:
    # Generate embeddings for the review using the Hugging Face model
    embedding = model.encode(review['review']).tolist()
    
    # Append the processed data
    processed_data.append({
        "values": embedding,
        "id": review['professor'],  
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })

# # Check the embedding dimension (optional)
# embedding_dim = len(processed_data[0]["values"])
# print("Embedding Dimension:", embedding_dim)  # Should print 1024


Embedding Dimension: 1024


In [12]:
processed_data[0]

{'values': [-0.014858714304864407,
  -0.015334726311266422,
  -0.01645059697329998,
  -0.03203741833567619,
  0.03843124955892563,
  -0.05087556689977646,
  -0.002738265786319971,
  0.034404903650283813,
  0.05430532619357109,
  -0.00876203179359436,
  0.027245167642831802,
  0.027922794222831726,
  -0.05334583669900894,
  -0.018113167956471443,
  -0.03263752534985542,
  0.007377806585282087,
  -0.010614569298923016,
  0.019906342029571533,
  0.022881120443344116,
  -0.03137194365262985,
  0.024258650839328766,
  0.0056487335823476315,
  -0.0722276046872139,
  -0.030151497572660446,
  -0.04640180990099907,
  -0.01283807773143053,
  -0.012951762415468693,
  -0.030907217413187027,
  -0.02746252343058586,
  -0.025642557069659233,
  -0.014613735489547253,
  -0.00493514072149992,
  -0.024883924052119255,
  -0.018170660361647606,
  -0.039902497082948685,
  0.020615387707948685,
  0.029962727800011635,
  0.019447339698672295,
  -0.030829349532723427,
  0.0305235106498003,
  -0.023377627134323

In [15]:
index = pc.Index('rag')
index.upsert(
    vectors= processed_data, #collection
    namespace= "ns1"   #document in firebase
)

{'upserted_count': 20}

In [16]:
index.describe_index_stats()

{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}